# Modeling

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
#
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

In [19]:
# Set device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Preparing Data

In [20]:
df_time_series = pd.read_csv('../../../data/df_monthly_returns_complete_percentage.csv', index_col='Date')

df_time_series = df_time_series.loc[:, ~df_time_series.columns.str.contains('^Unnamed')]

In [21]:
df_time_series = df_time_series

In [22]:
# 
df_time_series_plus1 = df_time_series
df_time_series = df_time_series - 1

### Normalisation

In [23]:
df_ts_torch = torch.from_numpy(df_time_series.values)
# Reshape to (num_samples, num_features) for normalization
df_ts_flat = df_ts_torch.view(-1, df_ts_torch.shape[-1])  # Shape: (1000*300, 5)
print(df_ts_flat)

# Calculate min and max per feature
df_min = df_ts_flat.min(dim=0, keepdim=True)[0]
df_max = df_ts_flat.max(dim=0, keepdim=True)[0]

# Apply Min-Max normalization
df_ts_normalised = (df_ts_flat - df_min) / (df_max - df_min)

# Reshape back to original shape
df_time_series_torch = df_ts_normalised.view(df_ts_torch.shape)


tensor([[    nan,     nan,     nan,  ...,     nan,     nan,     nan],
        [ 0.1300,  0.0000,  0.0000,  ...,  0.0000,  0.0000, -0.1600],
        [ 0.1100,  0.0000,  0.0000,  ...,  0.0000,  0.0000, -0.0700],
        ...,
        [ 0.1900,  0.0800,  0.0200,  ...,  0.2800, -0.1000,  0.0900],
        [-0.0400, -0.2200,  0.0600,  ..., -0.0700, -0.1100,  0.0400],
        [-0.0100, -0.0500,  0.1000,  ..., -0.0500, -0.0600, -0.0100]],
       dtype=torch.float64)


### Create Sequence

In [24]:

# Parameters
T = 12  # Sequence length (months)
num_tickers = len(df_time_series.columns)

# Preparing training data
sequences = []
targets = []

# Create sequences and targets
for i in range(len(df_time_series) - T):
    # Extract sequence of T months
    sequence = df_time_series.iloc[i:i + T].values  # Shape: (T, num_tickers)
    target = df_time_series.iloc[i + T].values      # Shape: (num_tickers)

    sequences.append(sequence)
    targets.append(target)

# Convert to numpy arrays
X = np.array(sequences)  # Shape: (number of sequences, T, num_tickers)
y = np.array(targets)     # Shape: (number of sequences, num_tickers)

# Print shapes of the training and target data
print("Training data shape (X):", X.shape)  # Should be (288, 12, 5)
print("Target data shape (y):", y.shape)     # Should be (288, 5)


Training data shape (X): (288, 12, 1653)
Target data shape (y): (288, 1653)


### Train-Test Split

In [25]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = (torch.tensor(X_train, dtype=torch.float32),
                                    torch.tensor(X_test, dtype=torch.float32),
                                    torch.tensor(y_train, dtype=torch.float32),
                                    torch.tensor(y_test, dtype=torch.float32))
# Check the shapes of the training and test data
print("Shape of X_train:", X_train.shape)  # Should be (230, 12, 5) for 80% of 288
print("Shape of y_train:", y_train.shape)  # Should be (230, 5)
print("Shape of X_test:", X_test.shape)    # Should be (58, 12, 5) for 20% of 288
print("Shape of y_test:", y_test.shape)    # Should be (58, 5)

Shape of X_train: torch.Size([230, 12, 1653])
Shape of y_train: torch.Size([230, 1653])
Shape of X_test: torch.Size([58, 12, 1653])
Shape of y_test: torch.Size([58, 1653])


### LSTM Model

In [26]:
# Define LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size=1, hidden_size=128, output_size=1):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=False)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        print('x', x.shape)
        lstm_out, _ = self.lstm(x)  # (seq_len, batch, hidden_size)
        final_output = lstm_out[:, -1, :]  # Last time step output
        print('pre final ', final_output.shape)
        print('final', self.fc(final_output).shape)
        return self.fc(final_output)  # (batch, output_size)

# Model, Loss, Optimizer
model = LSTMModel(input_size=len(df_time_series.columns), output_size=len(df_time_series.columns)).to(device)
criterion = nn.MSELoss()  # Mean Squared Error for regression
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [27]:
EPOCHS = 100
batch_size = 32

loss_fn = nn.MSELoss()
train_loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=False, batch_size=batch_size)
test_loader = data.DataLoader(data.TensorDataset(X_test, y_test), shuffle=False, batch_size=batch_size)

y_train_pred_all = torch.tensor([])
y_test_pred_all = torch.tensor([])

# Training Loop
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for index, (X_batch, y_batch) in enumerate(train_loader):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        if (torch.any(torch.isnan(X_batch)))|(torch.any(torch.isnan(y_batch))):
            #print("NaN values found during train in", epoch, i)
            continue
        # Forward pass
        optimizer.zero_grad()
        y_pred = model(X_batch)
        y_pred = y_pred.squeeze(-1)  # @TODO check here - Remove last dim for (batch, 1653)

        # Compute loss
        loss = criterion(y_pred, y_batch)
        total_loss += loss.item()

        # Backward pass
        loss.backward()
        optimizer.step()
        # Join all batch predictions together - only last epoch where model is fully trained
        if epoch == EPOCHS - 1:
            y_train_pred_all = torch.cat([y_train_pred_all, y_pred], dim=0)

    # Validation - Root-mean-square-error
    # if epoch != 1 and epoch % 100 != 0:
    #    continue
    model.eval()
    with torch.no_grad():
        for index, (X_batch, y_batch) in enumerate(test_loader):
            #print('test', X_batch.shape, y_batch.shape)
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            if (torch.any(torch.isnan(X_batch)))|(torch.any(torch.isnan(y_batch))):
                print("NaN values found during test ", epoch, i)
                continue
                y_test_pred = model(X_batch)
                y_test_pred = y_test_pred.squeeze(-1)  # @TODO check here - Remove last dim for (batch, 1653)
                #print(y_test_pred)
            # Join all batch predictions together
            if epoch == EPOCHS - 1:
                y_test_pred_all = torch.cat([y_test_pred_all, y_test_pred], dim=0)
        
        #
        y_train_pred = model(X_train)
        train_rmse = np.sqrt(loss_fn(y_train_pred, y_train))
        y_test_pred = model(X_test)
        test_rmse = np.sqrt(loss_fn(y_test_pred, y_test))
    # Print epoch loss
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {total_loss / len(train_loader):.6f}")

# Save the trained model
torch.save(model.state_dict(), "lstm_univariate.pth")
print("Model training complete and saved.")

x torch.Size([32, 12, 1653])
pre final  torch.Size([32, 128])
final torch.Size([32, 1653])
x torch.Size([32, 12, 1653])
pre final  torch.Size([32, 128])
final torch.Size([32, 1653])
x torch.Size([32, 12, 1653])
pre final  torch.Size([32, 128])
final torch.Size([32, 1653])
x torch.Size([32, 12, 1653])
pre final  torch.Size([32, 128])
final torch.Size([32, 1653])
x torch.Size([32, 12, 1653])
pre final  torch.Size([32, 128])
final torch.Size([32, 1653])
x torch.Size([32, 12, 1653])
pre final  torch.Size([32, 128])
final torch.Size([32, 1653])
x torch.Size([6, 12, 1653])
pre final  torch.Size([6, 128])
final torch.Size([6, 1653])
x torch.Size([230, 12, 1653])
pre final  torch.Size([230, 128])
final torch.Size([230, 1653])
x torch.Size([58, 12, 1653])
pre final  torch.Size([58, 128])
final torch.Size([58, 1653])
Epoch 1/100, Loss: 0.123132
x torch.Size([32, 12, 1653])
pre final  torch.Size([32, 128])
final torch.Size([32, 1653])
x torch.Size([32, 12, 1653])
pre final  torch.Size([32, 128])


### Returns vs Predicted

In [28]:
# Compute average portfolio returns over all assets (per time step)
true_avg = pd.DataFrame(torch.tensor(y, dtype=torch.float32)).mean(axis=1)
pred_train_avg = pd.DataFrame(pd.DataFrame(y_train_pred_all.detach().numpy())).mean(axis=1)
pred_test_avg = pd.DataFrame(pd.DataFrame(y_test_pred_all.detach().numpy())).mean(axis=1)

'''pred_avg = pred_avg.reindex(range(len(true_avg)))

# Time indices
time_steps = np.arange(len(df_time_series))
print(len(pred_avg))
table = pd.DataFrame( {"Predicted returns": pred_avg.tolist(), "Actual returns": true_avg.tolist()})
table'''

'pred_avg = pred_avg.reindex(range(len(true_avg)))\n\n# Time indices\ntime_steps = np.arange(len(df_time_series))\nprint(len(pred_avg))\ntable = pd.DataFrame( {"Predicted returns": pred_avg.tolist(), "Actual returns": true_avg.tolist()})\ntable'

In [29]:
# Plotly Visualization
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_time_series.index.tolist(), y=true_avg, mode='lines', name='Actual Returns',
                         line=dict(color='#5c839f', width=2)))

# Add the training plot in red
fig.add_trace(go.Scatter(x=df_time_series.index.tolist(), y=pred_train_avg, mode='lines', name='Train returns',
                         line=dict(color='red', width=2)))

fig.add_trace(go.Scatter(x=df_time_series.index.tolist()[len(pred_train_avg):], y=pred_test_avg, mode='lines', name='Predicted Returns',
                         line=dict(color='green')))

# Layout settings
fig.update_layout(
    title="Portfolio Monthly Returns: Predicted vs Actual",
    legend_title="Legend",
    template="plotly_white",
    xaxis=dict(
        title='Date'
    ),
    yaxis=dict(
        title='Average Monthly Portfolio Return (%)',
        tickformat='.0%',
        range=[-0.2,0.2]
    ),
    legend=dict(title="Legend")
)

# Show plot
fig.show()

In [30]:
y_test_pred_all

tensor([[-0.0090,  0.0238,  0.0289,  ...,  0.0388, -0.0187, -0.0072],
        [-0.0218,  0.0447,  0.0130,  ..., -0.0566, -0.0092,  0.0076],
        [-0.0535,  0.0345,  0.0932,  ...,  0.0212,  0.0850,  0.0602],
        ...,
        [-0.1085,  0.0602, -0.0728,  ..., -0.0161,  0.0313,  0.0490],
        [-0.1030,  0.1579, -0.2251,  ...,  0.1269,  0.1567,  0.0316],
        [-0.0747,  0.1096, -0.2117,  ...,  0.3434,  0.0146, -0.0185]])

## Sharpe Ratio

### Prediction to Dataframe

In [31]:
y_test_pred_all = y_test_pred_all + 1
#
df_pred = pd.DataFrame(y_test_pred_all)
df_pred.columns = df_time_series.columns
df_pred

,RS1.L,KE,TEG.DE,LEG.DE,SCS,HNI,AVT,ACCO,VNA.DE,7912.T,...,DEQ.DE,KIDS,HALO,MATW,9842.T,KVHI,MOON.L,NEO,6055.T,UNP
0,0.990989,1.023785,1.028870,0.989278,0.990400,0.996975,1.017439,0.961675,0.994879,1.026307,...,1.037436,0.998356,1.039426,1.012241,0.966799,1.085997,1.009112,1.038763,0.981328,0.992790
1,0.978167,1.044712,1.012977,0.997082,1.041944,1.028689,1.010759,1.058331,1.020771,1.012241,...,1.005557,1.015309,1.167767,0.996894,1.030653,1.070280,1.014827,0.943358,0.990772,1.007560
2,0.946455,1.034482,1.093150,0.972525,1.030048,1.101226,1.069786,1.085241,0.996851,1.051902,...,0.970529,0.890533,1.296280,0.993726,0.990663,1.181107,1.049018,1.021234,1.084969,1.060204
3,1.051320,1.074293,0.917925,0.927361,1.141613,1.053242,0.964781,0.735844,0.954203,0.841425,...,1.110563,1.024342,0.943165,0.939360,1.007910,1.124603,0.989284,1.460597,0.901228,0.928714
4,0.993328,1.139612,1.090143,0.968633,1.099076,1.174962,1.196253,1.124121,0.968330,0.958854,...,1.129895,1.043117,1.489024,0.983299,1.055666,1.058549,0.921472,0.889705,0.823400,1.143525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,0.979265,1.057094,1.039922,0.974521,1.026218,1.092619,1.060649,1.065529,0.990664,1.000797,...,0.958593,0.994800,0.879463,1.038225,0.999648,1.082705,0.919325,0.926216,1.034012,1.048195
112,0.918245,1.090255,1.105510,0.938637,1.048455,0.960294,1.006520,0.801641,0.932481,1.110362,...,1.012471,1.129356,0.854291,1.045289,1.132668,1.259657,0.925729,0.689885,1.082048,1.053514
113,0.891491,1.060163,0.927160,0.946828,0.994486,1.032922,1.051780,0.903102,0.909668,1.066817,...,0.967029,1.056186,1.082348,1.092377,1.019271,1.108406,0.912549,0.983932,1.031302,1.048974
114,0.896960,1.157941,0.774855,0.915014,1.124650,1.153954,1.186766,0.675427,0.875425,1.072200,...,0.977007,1.165247,1.347144,1.150930,1.142614,1.222854,0.947258,1.126945,1.156665,1.031555


In [32]:
def build_efficient_frontier(df_pred):
    # Calculate expected returns and sample covariance
    mu_0 = expected_returns.mean_historical_return(df_pred, frequency=12)
    # Get only tickers with a mean historical return of at least 5% 
    optimal_tickers = mu_0[mu_0 > 0.05].index

    df_optimal = df_pred[optimal_tickers]
    
    mu = expected_returns.mean_historical_return(df_optimal)
    S = risk_models.CovarianceShrinkage(df_optimal).ledoit_wolf() # Exponential Covariance

    # Optimize for maximal Sharpe ratio
    ef = EfficientFrontier(mu, S)
    ef_new = EfficientFrontier(mu, S)

    raw_weights = ef.max_sharpe()
    cleaned_weights = ef.clean_weights()
    ef.save_weights_to_file("weights.csv")  # saves to file
    #
    ef.portfolio_performance(verbose=True)

    return df_optimal
# @TODO to check results - df_pred is just the test results, ok?
build_efficient_frontier( df_pred )

Expected annual return: 149068.7%
Annual volatility: 5170.1%
Sharpe Ratio: 28.83


,IVT,ILM1.DE,MCG.L
0,0.984336,1.012130,0.126373
1,0.896552,1.675529,1.226264
2,-0.661052,1.651364,3.267381
3,0.379736,1.712559,-0.407437
4,0.208354,1.829538,-0.307377
...,...,...,...
111,0.925157,1.221168,-1.567851
112,3.243841,1.165047,-2.747906
113,1.748227,1.116440,1.158442
114,1.609294,1.855105,7.123019


### Optimal Allocation